# BioLLM x Plants - Procedure Design - Multi-Agent

Rachel K. Luu, Ming Dao, Subra Suresh, Markus J. Buehler (2025) [full reference to be updated to be included here]


# Load Packages

In [ ]:
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex,Settings,PromptTemplate
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from utils.formats import messages_to_prompt, completion_to_prompt

# Load BioLLM

In [ ]:
model_url = "https://huggingface.co/rachelkluu/Llama3.1-8b-Instruct-CPT-SFT-DPO-09022024-Q4_K_M-GGUF/resolve/main/llama3.1-8b-instruct-cpt-sft-dpo-09022024-q4_k_m.gguf"

biollm_agent = LlamaCPP(
    model_url=model_url,
    model_path=None,
    temperature=0.1,
    max_new_tokens=1024,
    context_window=10000,
    
    generate_kwargs={"top_k":256, "top_p":0.9,"repeat_penalty":1.1,},
     
    model_kwargs={"n_gpu_layers": -1
                 },

    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

# Load Llama3.1

In [3]:
model_url= "https://huggingface.co/rachelkluu/Llama-3.1-8B-Instruct-Q4_K_M-GGUF/resolve/main/llama-3.1-8b-instruct-q4_k_m.gguf"

llm_question = LlamaCPP(
    model_url=model_url,
    model_path=None,
    temperature=0.1,
    max_new_tokens=1024,
    context_window=5000,
    generate_kwargs={"top_k":256, "top_p":0.9,"repeat_penalty":1.1,},
     
    model_kwargs={"n_gpu_layers": -1,
                 },
   messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=False,
)

# Load RAG For LLM Agents

In [5]:
data = SimpleDirectoryReader(input_dir="./RAG/",).load_data()

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_documents(
    data, transformations=[SentenceSplitter(chunk_size=256),],verbose=True, embed_model = embed_model ,
) 

# Load JSON Q-A File

In [6]:
import json

output_string = ""
jsonfile = "rhapispollencomp" #name of saved  json file with generated Q-As

with open(f'{jsonfile}.json', 'r', encoding='utf-8') as file:
    for line in file:
        data = json.loads(line)
        question = data.get('Question', 'No question found')
        answer = data.get('Answer', 'No answer found')
        output_string += f"Question: {question}\nAnswer: {answer}\n\n"

print(output_string)


Question: What are the unique properties of pollen grains and rhapis excelsa leaves?
Answer: Pollen grains have unique properties such as a hollow internal architecture, uniform microscale size, and robust hierarchical structure. They are abundant in nature and harvested by bees. Sunflower pollen grains have a distinctive core-shell structure with spiky architectures, with the shell comprising an ultra-strong sporopollenin outer layer and a flexible inner layer consisting of cellulose and pectin. Rhapis excelsa leaves have a unique hierarchical structure with a high density of longitudinal veins and a distinct pattern of transverse veins.

Question: What are the advantages of using pollen grains and rhapis excelsa leaves in a composite material?
Answer: The advantages of using pollen grains and rhapis excelsa leaves in a composite material include their unique hierarchical structures, which contribute to the material's mechanical properties such as high strength and toughness. The comb

# Conversation

In [8]:
from protocols.proc_agents import make_dir_if_needed, answer_question_LlamaIndex, remove_markdown_symbols
from IPython.display import Markdown, display

data_dir='./convo_out/'
make_dir_if_needed(data_dir)

prompt = "Design a procedure that makes a composite out of pollen grains and rhapis excelsa leaves."
finalquery = f"""{prompt}

Here is some additional information that can support this task: 
{output_string}"""

##########################################
# First bot is the expert being asked
##########################################
bot_name_1="BioinspiredLLM"
bot_instructions_1 = f"""You are a {bot_name_1}, a creative engineer with knowledge in biological and bio-inspired materials.

You are taking part in a discussion and have access to potentially relevant information from previous papers on Rhapis excelsa leaves and pollen-based materials. 

However, be cautious and do not cite specific studies or references. Focus on providing concise, accurate, and thoughtful contributions that stimulate new trains of thought.
"""
##########################################
# Second bot asks questions in an adversarial way
##########################################
bot_name_2="Scientist"
bot_instructions_2 = f"""You are a {bot_name_2}, taking part in a discussion where your role is to collaborate with BioinspiredLLM to develop a scientific procedure.

Prioritize scientific effectivenes and logical reasoning when providing potential implications or improvements.

You have access to potentially relevant information from previous papers on Rhapis excelsa leaves and pollen-based materials. """

sample_question=''

total_turns=2
include_N_turns_in_question_development=total_turns
##########################################
# Let's begin
##########################################
conversation_turns, txt, summary, integrated, save_raw_txt, answer_agent_conv, response, answer_agent = answer_question_LlamaIndex (# model, tokenizer,
                                                                                                            q=finalquery,
                llm_answer=biollm_agent,
                    llm_question=llm_question,  
                        llm_summarize=llm_question,
                    bot_name_1=bot_name_1,
                    bot_instructions_1 = bot_instructions_1,
                    bot_name_2=bot_name_2,
                    bot_instructions_2 = bot_instructions_2,
                    total_turns=total_turns,
                    delete_last_question=True,
                    save_dir=data_dir, marker_ch='>>>',marker_ch_outer='###',
    start_with_q=False,only_last=False,
        question_index=index, answer_index=index,
    verbose=False, 
    chat_mode="context",chat_token_limit=5000,single_shot_question=True,
    sample_question=sample_question,include_N_turns_in_question_development=include_N_turns_in_question_development,
                    )

display(Markdown(integrated))
txt=remove_markdown_symbols(integrated)
max_len_fname=64
txt_file_path = f'{data_dir}{finalquery[:max_len_fname].strip()}.txt'
save_raw_txt=remove_markdown_symbols(integrated)

with open(txt_file_path, 'w') as file:
    file.write(save_raw_txt)

Set up chat engine, with index, verbose=False, chat_mode=context.
### Scientist: Design a procedure that makes a composite out of pollen grains and rhapis excelsa leaves.

Here is some additional information that can support this task: 
Question: What are the unique properties of pollen grains and rhapis excelsa leaves?
Answer: Pollen grains have unique properties such as a hollow internal architecture, uniform microscale size, and robust hierarchical structure. They are abundant in nature and harvested by bees. Sunflower pollen grains have a distinctive core-shell structure with spiky architectures, with the shell comprising an ultra-strong sporopollenin outer layer and a flexible inner layer consisting of cellulose and pectin. Rhapis excelsa leaves have a unique hierarchical structure with a high density of longitudinal veins and a distinct pattern of transverse veins.

Question: What are the advantages of using pollen grains and rhapis excelsa leaves in a composite material?
Answer:

Llama.generate: 1199 prefix-match hit, remaining 1 prompt tokens to eval


Procedure for Making a Composite Out of Pollen Grains and Rhapis Excelsa Leaves:

1. Harvesting: Collect sunflower pollen grains and rhapis excelsa leaves from their respective sources.
2. Cleaning: Clean the harvested materials by removing any impurities, such as dirt or debris.
3. Preparation: Prepare the matrix material for the composite. This can be done using various methods, such as solution casting, melt mixing, or electrospinning.
4. Mixing: Mix the pollen grains and rhapis excelsa leaves with the matrix material to create a homogeneous mixture.
5. Forming: Form the mixture into the desired shape and size of the composite material.
6. Drying: Allow the composite material to dry completely before further processing, if necessary.
7. Characterization: Characterize the mechanical properties of the composite material using various testing methods, such as tensile strength, flexural strength, or impact resistance.

Note: The specific steps for each stage may vary depending on the ch

Llama.generate: 1433 prefix-match hit, remaining 409 prompt tokens to eval


Thank you for your valuable suggestions on refining the procedure for making a composite out of pollen grains and rhapis excelsa leaves. Here is an updated version of the procedure that incorporates these refinements:

Procedure for Making a Composite Out of Pollen Grains and Rhapis Excelsa Leaves:

1. Harvesting: Collect sunflower pollen grains and rhapis excelsa leaves from their respective sources.
2. Cleaning: Clean the harvested materials by removing any impurities, such as dirt or debris.
3. Surface modification: Modify the surfaces of the pollen grains and rhapis excelsa leaves to enhance interfacial bonding between the components. This can be done using various methods, such as chemical grafting, plasma treatment, or silane coupling agents.
4. Matrix material preparation: Prepare the matrix material for the composite. This can be done using various methods, such as solution casting, melt mixing, or electrospinning. Specify the exact type of polymer, metal, or ceramic being used

#### Question and conversation:
    
The question discussed is: **Design a procedure that makes a composite out of pollen grains and rhapis excelsa leaves.

Here is some additional information that can support this task: 
Question: What are the unique properties of pollen grains and rhapis excelsa leaves?
Answer: Pollen grains have unique properties such as a hollow internal architecture, uniform microscale size, and robust hierarchical structure. They are abundant in nature and harvested by bees. Sunflower pollen grains have a distinctive core-shell structure with spiky architectures, with the shell comprising an ultra-strong sporopollenin outer layer and a flexible inner layer consisting of cellulose and pectin. Rhapis excelsa leaves have a unique hierarchical structure with a high density of longitudinal veins and a distinct pattern of transverse veins.

Question: What are the advantages of using pollen grains and rhapis excelsa leaves in a composite material?
Answer: The advantages of using pollen grains and rhapis excelsa leaves in a composite material include their unique hierarchical structures, which contribute to the material's mechanical properties such as high strength and toughness. The combination of these natural materials can potentially enhance the overall performance of the composite, making it suitable for various applications. However, further exploration is needed to quantify and characterize the specific advantages of this composite material.

Question: What are the potential applications of a composite material made from pollen grains and rhapis excelsa leaves?
Answer: The potential applications of a composite material made from pollen grains and rhapis excelsa leaves include sustainable and biodegradable materials for various engineering and biomedical applications, such as shape-memory cryogels for treating noncompressible hemorrhages, and eco-friendly derivative materials for different applications. The unique properties of pollen and rhapis excelsa leaves, such as their hollow internal architecture, uniform microscale size, and robust hierarchical structure, make them promising candidates for developing novel classes of sustainable materials. However, further exploration is needed to fully understand the properties and potential applications of this composite material.

Question: What are the challenges in making a composite material from pollen grains and rhapis excelsa leaves?
Answer: The challenges in making a composite material from pollen grains and rhapis excelsa leaves include the need for a compatible interface between the two components, as well as the differences in their mechanical properties and structural characteristics. The unique properties of pollen and rhapis excelsa leaves, such as their hollow microsphere structures and components, require a tailored approach for their integration into a composite material. Further exploration is needed to address these challenges and develop a successful composite material.

Question: What are pollen grains and rhapis excelsa leaves?
Answer: Pollen grains are abundant natural resources with unique properties such as a hollow internal architecture, uniform microscale size, and robust hierarchical structure. Rhapis excelsa leaves are a type of palm tree native to Southeast Asia, known for their distinctive fan-shaped leaves and ability to purify the air.

Question: What is the process of making a composite material?
Answer: The process of making a composite material involves combining two or more materials with different properties to create a new material with improved properties. This can be achieved through various methods such as mixing, layering, or embedding. The materials used in composite manufacturing can be natural or synthetic, and they can be in the form of fibers, particles, or sheets. The properties of the composite material depend on the properties of the individual components and their interaction with each other. The process of making a composite material can be tailored to achieve specific properties and applications, such as strength, durability, flexibility, or conductivity.

Question: What is a composite material?
Answer: A composite material is a material made from two or more constituent materials with different physical or chemical properties, which, when combined, create a material with unique properties that are different from those of its individual components.

Question: What are the different methods of making a composite material?
Answer: The different methods of making a composite material include:
1. Layer-by-layer (LBL) assembly: Alternating deposition of layers of different materials to create a composite.
2. Sol-gel processing: A chemical process that involves the hydrolysis and condensation of metal or metal oxide precursors to form a gel, which is then dried and calcined to form a composite.
3. Melt mixing: Mixing two or more materials in a molten state to create a composite.
4. Solution casting: Dissolving the components of the composite in a solvent and casting the solution to form a composite.
5. Fiber-reinforced composite processing: Incorporating fibers into a matrix material to create a composite.
6. Freeze casting: A process that involves freezing a solution containing the components of the composite, followed by the removal of the solvent and the formation of a composite.
7. Electrospinning: A process that involves the electrostatic spinning of fibers from a solution, which can be used to create a composite.
8. 3D printing: A process that involves the layer-by-layer deposition of materials to create a composite with complex geometries.
9. Coating: Applying a layer of one material onto another to create a composite.
10. Chemical vapor deposition (CVD): A process that involves the deposition of a material onto a substrate using a gas-phase reaction.

Question: What are the essential components of a composite material?
Answer: The essential components of a composite material are a matrix and one or more reinforcement materials. The matrix is the continuous phase that surrounds and supports the reinforcement, while the reinforcement is the discontinuous phase that provides strength and stiffness to the composite. The matrix can be a polymer, metal, or ceramic, while the reinforcement can be fibers, particles, or other shapes. The combination of the matrix and reinforcement materials results in a material with improved mechanical properties compared to the individual components.**

**Scientist**: Design a procedure that makes a composite out of pollen grains and rhapis excelsa leaves.

Here is some additional information that can support this task: 
Question: What are the unique properties of pollen grains and rhapis excelsa leaves?
Answer: Pollen grains have unique properties such as a hollow internal architecture, uniform microscale size, and robust hierarchical structure. They are abundant in nature and harvested by bees. Sunflower pollen grains have a distinctive core-shell structure with spiky architectures, with the shell comprising an ultra-strong sporopollenin outer layer and a flexible inner layer consisting of cellulose and pectin. Rhapis excelsa leaves have a unique hierarchical structure with a high density of longitudinal veins and a distinct pattern of transverse veins.

Question: What are the advantages of using pollen grains and rhapis excelsa leaves in a composite material?
Answer: The advantages of using pollen grains and rhapis excelsa leaves in a composite material include their unique hierarchical structures, which contribute to the material's mechanical properties such as high strength and toughness. The combination of these natural materials can potentially enhance the overall performance of the composite, making it suitable for various applications. However, further exploration is needed to quantify and characterize the specific advantages of this composite material.

Question: What are the potential applications of a composite material made from pollen grains and rhapis excelsa leaves?
Answer: The potential applications of a composite material made from pollen grains and rhapis excelsa leaves include sustainable and biodegradable materials for various engineering and biomedical applications, such as shape-memory cryogels for treating noncompressible hemorrhages, and eco-friendly derivative materials for different applications. The unique properties of pollen and rhapis excelsa leaves, such as their hollow internal architecture, uniform microscale size, and robust hierarchical structure, make them promising candidates for developing novel classes of sustainable materials. However, further exploration is needed to fully understand the properties and potential applications of this composite material.

Question: What are the challenges in making a composite material from pollen grains and rhapis excelsa leaves?
Answer: The challenges in making a composite material from pollen grains and rhapis excelsa leaves include the need for a compatible interface between the two components, as well as the differences in their mechanical properties and structural characteristics. The unique properties of pollen and rhapis excelsa leaves, such as their hollow microsphere structures and components, require a tailored approach for their integration into a composite material. Further exploration is needed to address these challenges and develop a successful composite material.

Question: What are pollen grains and rhapis excelsa leaves?
Answer: Pollen grains are abundant natural resources with unique properties such as a hollow internal architecture, uniform microscale size, and robust hierarchical structure. Rhapis excelsa leaves are a type of palm tree native to Southeast Asia, known for their distinctive fan-shaped leaves and ability to purify the air.

Question: What is the process of making a composite material?
Answer: The process of making a composite material involves combining two or more materials with different properties to create a new material with improved properties. This can be achieved through various methods such as mixing, layering, or embedding. The materials used in composite manufacturing can be natural or synthetic, and they can be in the form of fibers, particles, or sheets. The properties of the composite material depend on the properties of the individual components and their interaction with each other. The process of making a composite material can be tailored to achieve specific properties and applications, such as strength, durability, flexibility, or conductivity.

Question: What is a composite material?
Answer: A composite material is a material made from two or more constituent materials with different physical or chemical properties, which, when combined, create a material with unique properties that are different from those of its individual components.

Question: What are the different methods of making a composite material?
Answer: The different methods of making a composite material include:
1. Layer-by-layer (LBL) assembly: Alternating deposition of layers of different materials to create a composite.
2. Sol-gel processing: A chemical process that involves the hydrolysis and condensation of metal or metal oxide precursors to form a gel, which is then dried and calcined to form a composite.
3. Melt mixing: Mixing two or more materials in a molten state to create a composite.
4. Solution casting: Dissolving the components of the composite in a solvent and casting the solution to form a composite.
5. Fiber-reinforced composite processing: Incorporating fibers into a matrix material to create a composite.
6. Freeze casting: A process that involves freezing a solution containing the components of the composite, followed by the removal of the solvent and the formation of a composite.
7. Electrospinning: A process that involves the electrostatic spinning of fibers from a solution, which can be used to create a composite.
8. 3D printing: A process that involves the layer-by-layer deposition of materials to create a composite with complex geometries.
9. Coating: Applying a layer of one material onto another to create a composite.
10. Chemical vapor deposition (CVD): A process that involves the deposition of a material onto a substrate using a gas-phase reaction.

Question: What are the essential components of a composite material?
Answer: The essential components of a composite material are a matrix and one or more reinforcement materials. The matrix is the continuous phase that surrounds and supports the reinforcement, while the reinforcement is the discontinuous phase that provides strength and stiffness to the composite. The matrix can be a polymer, metal, or ceramic, while the reinforcement can be fibers, particles, or other shapes. The combination of the matrix and reinforcement materials results in a material with improved mechanical properties compared to the individual components.



**BioinspiredLLM**: Procedure for Making a Composite Out of Pollen Grains and Rhapis Excelsa Leaves:

1. Harvesting: Collect sunflower pollen grains and rhapis excelsa leaves from their respective sources.
2. Cleaning: Clean the harvested materials by removing any impurities, such as dirt or debris.
3. Preparation: Prepare the matrix material for the composite. This can be done using various methods, such as solution casting, melt mixing, or electrospinning.
4. Mixing: Mix the pollen grains and rhapis excelsa leaves with the matrix material to create a homogeneous mixture.
5. Forming: Form the mixture into the desired shape and size of the composite material.
6. Drying: Allow the composite material to dry completely before further processing, if necessary.
7. Characterization: Characterize the mechanical properties of the composite material using various testing methods, such as tensile strength, flexural strength, or impact resistance.

Note: The specific steps for each stage may vary depending on the chosen method and desired application of the composite material. Further exploration is needed to optimize the process and fully understand the properties and potential applications of this composite material.

**Scientist**: To improve the scientific validity of the procedure, I would suggest the following refinements:

1. **Add a step for surface modification**: Before mixing the pollen grains and rhapis excelsa leaves with the matrix material, consider adding a step to modify their surfaces to enhance interfacial bonding between the components.
2. **Specify the type of matrix material**: Instead of using general terms like matrix material, specify the exact type of polymer, metal, or ceramic being used as the matrix.
3. **Include a step for optimizing the ratio of pollen grains to rhapis excelsa leaves**: The optimal ratio of these components may vary depending on the desired application and properties of the composite material. Consider adding a step to determine this ratio through experimentation.
4. **Mention the need for mechanical testing under various conditions**: While tensile strength, flexural strength, and impact resistance are important mechanical properties, consider mentioning the need to test the composite material under different environmental conditions (e.g., temperature, humidity) to simulate real-world applications.
5. **Add a step for characterizing the composite's thermal and electrical properties**: Depending on the intended application, it may be essential to characterize the composite's thermal conductivity, electrical conductivity, or other relevant properties.
6. **Consider using advanced characterization techniques**: While tensile testing is a common method for evaluating mechanical properties, consider mentioning the use of more advanced techniques like atomic force microscopy (AFM), scanning electron microscopy (SEM), or transmission electron microscopy (TEM) to study the composite's microstructure and interface.
7. **Emphasize the importance of controlling the processing conditions**: The processing conditions, such as temperature, pressure, and time, can significantly impact the final properties of the composite material. Consider adding a step to optimize these conditions through experimentation.

By incorporating these refinements, the procedure will be more comprehensive and scientifically valid, allowing researchers to create high-quality composites with tailored properties for specific applications.

**BioinspiredLLM**: Thank you for your valuable suggestions on refining the procedure for making a composite out of pollen grains and rhapis excelsa leaves. Here is an updated version of the procedure that incorporates these refinements:

Procedure for Making a Composite Out of Pollen Grains and Rhapis Excelsa Leaves:

1. Harvesting: Collect sunflower pollen grains and rhapis excelsa leaves from their respective sources.
2. Cleaning: Clean the harvested materials by removing any impurities, such as dirt or debris.
3. Surface modification: Modify the surfaces of the pollen grains and rhapis excelsa leaves to enhance interfacial bonding between the components. This can be done using various methods, such as chemical grafting, plasma treatment, or silane coupling agents.
4. Matrix material preparation: Prepare the matrix material for the composite. This can be done using various methods, such as solution casting, melt mixing, or electrospinning. Specify the exact type of polymer, metal, or ceramic being used as the matrix.
5. Mixing and optimization: Mix the pollen grains and rhapis excelsa leaves with the matrix material to create a homogeneous mixture. Determine the optimal ratio of these components through experimentation based on the desired application and properties of the composite material.
6. Forming: Form the mixture into the desired shape and size of the composite material.
7. Drying: Allow the composite material to dry completely before further processing, if necessary.
8. Mechanical testing under various conditions: Test the composite material's mechanical properties (e.g., tensile strength, flexural strength, impact resistance) under different environmental conditions (e.g., temperature, humidity) to simulate real-world applications.
9. Characterization of thermal and electrical properties: Characterize the composite's thermal conductivity, electrical conductivity, or other relevant properties depending on the intended application.
10. Advanced characterization techniques: Use advanced characterization techniques like atomic force microscopy (AFM), scanning electron microscopy (SEM), or transmission electron microscopy (TEM) to study the composite's microstructure and interface.
11. Optimization of processing conditions: Optimize the processing conditions (e.g., temperature, pressure, time) through experimentation to ensure high-quality composites with tailored properties for specific applications.

By incorporating these refinements, the procedure will be more comprehensive and scientifically valid, allowing researchers to create high-quality composites with tailored properties for specific applications. <eot_id>

 

#### Summary:

**Final Refined Procedure:**

1. **Harvesting and Cleaning**: Collect sunflower pollen grains and rhapis excelsa leaves from their respective sources, and clean them by removing any impurities.
2. **Surface Modification**: Modify the surfaces of the pollen grains and rhapis excelsa leaves to enhance interfacial bonding between the components using methods such as chemical grafting or plasma treatment.
3. **Matrix Material Preparation**: Prepare a polymer matrix material (e.g., polyethylene, polypropylene) for the composite using solution casting or melt mixing.
4. **Mixing and Optimization**: Mix the pollen grains and rhapis excelsa leaves with the matrix material to create a homogeneous mixture. Determine the optimal ratio of these components through experimentation based on the desired application and properties of the composite material.
5. **Forming**: Form the mixture into the desired shape and size of the composite material using techniques such as injection molding or 3D printing.
6. **Drying**: Allow the composite material to dry completely before further processing, if necessary.
7. **Mechanical Testing under Various Conditions**: Test the composite material's mechanical properties (e.g., tensile strength, flexural strength, impact resistance) under different environmental conditions (e.g., temperature, humidity) to simulate real-world applications.
8. **Characterization of Thermal and Electrical Properties**: Characterize the composite's thermal conductivity, electrical conductivity, or other relevant properties depending on the intended application.
9. **Advanced Characterization Techniques**: Use advanced characterization techniques like atomic force microscopy (AFM), scanning electron microscopy (SEM), or transmission electron microscopy (TEM) to study the composite's microstructure and interface.

**Note:** The specific steps for each stage may vary depending on the chosen method and desired application of the composite material. Further exploration is needed to optimize the process and fully understand the properties and potential applications of this composite material.

